In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, GRU, MaxPooling1D, Conv1D, Flatten
from keras.layers import LSTM, Dropout, Bidirectional, GlobalMaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:

# lets explore the dataset
train_data = pd.read_csv(train_file_path, delimiter='\t')
test_data = pd.read_csv(test_file_path, delimiter='\t')
# rename the column of text so is easier to work
train_data.rename(columns ={'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.': 'sms'}, inplace = True)
test_data.rename(columns ={'i am in hospital da. . i will return home in evening': 'sms'}, inplace = True)

# now we need to get the categorical data in ham into binary
train_label = train_data.pop('ham').map({'ham': 0, 'spam': 1})
test_label = test_data.pop('ham').map({'ham':0, 'spam':1})

MAXLEN = 300
BATCH_SIZE = 64



In [ ]:
# we use the tokenizer function to turn the data into something
# the model can use
def tok_data(top_info):
  token = Tokenizer(num_words = 7691)
  token.fit_on_texts(top_info)
  train_sec = token.texts_to_sequences(top_info)
  train_pad = pad_sequences(train_sec, maxlen=MAXLEN).astype(np.float32)
  return train_pad
train_data = tok_data(train_data['sms'])
test_data = tok_data(test_data['sms'])

#we have to pass the label dataframe as a np.array
train_label = (train_label).astype('float32')
test_label = (test_label).astype('float32')
print(test_data.shape)


In [ ]:
# now we can create the model
# Create Sequential model
model = tf.keras.Sequential([
    Embedding(input_dim = 7691, output_dim =128, input_length = MAXLEN),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate= 0.0001), metrics=['accuracy'])
model.summary()



In [ ]:
# then we need to fit the model
# it was necessary to implement an early stop function
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(train_pad, train_label, epochs=15, validation_data=(test_pad, test_label), callbacks = [early_stop])

In [ ]:
loss,accuracy = model.evaluate(test_pad, test_label)
print(f'Loss:{loss}, Precission:{accuracy}')

In [ ]:
pred_text = "how are you doing today?"
pred_text_pad = tok_data(pred_text)
prediction = model.predict(pred_text_pad)
print(f'Prediction:{prediction}')
predicted = np.argmax(prediction, axis = 1)
labels = ['ham', 'spam']
resultado = labels[predicted[0]]
print(f'Predicted:{resultado}')

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  if isinstance(pred_text, str):
    pred_text = [pred_text]
  prep_text = tok_data(pred_text)
  prediction_np = model.predict(prep_text)
  if (prediction_np[0]*100) > 0.5:
    prediction = 'spam'
  else:
    prediction = 'ham'
  return (prediction_np[0],prediction)

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
